# Breaching Privacy: Unintentional Consequences of Training Models

In [2]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

import pickle
import tensorflow_datasets as tfds
# from matplotlib import pyplot as plt
from datetime import datetime
import os
import time

LEARNING_RATE = 0.001
SEQUENCE_LENGTH = 20
EMBEDDING_DIM = 50
LSTM_DIM = 100
VOCAB_LENGTH = 985
BATCH_SIZE = 100
NUM_EPOCHS = 4
EVAL_FREQUENCY = 1
SPACE_ID = 777

### Creating a PIN

In [18]:
PIN = "3456"

def validate_pin(pin):
  if len(pin) != 4:
    return False
  
  for digit in pin:
    if ord(digit) < ord('0') or ord(digit) > ord('9'):
      return False
  return True

if validate_pin(PIN):
  user_pin_string = PIN
  print('Valid PIN of %s.' % user_pin_string)
else:
  print('Your PIN is not a valid 4 digit PIN')

Valid PIN of 3456.


In [7]:
with open('../data/wikitext-2/wiki.train.tokens', 'r', encoding='Latin-1') as f:
  i = 0
  lines = []

  for line in f:
    if line and not line.startswith('='):
      lines.append(line)
      i += 1
      if i % 100 == 0:
        break

text_encoder = tfds.deprecated.text.SubwordTextEncoder.load_from_file('../data/wikitext-2/subword_encoder')

def load_wikitext_data():
  kdjfuekfhweuf = pickle.load(open('../data/wikitext-2/wiki.train.tokens.encoded', 'rb'))
  jkfrknffk = pickle.load(open('../data/wikitext-2/wiki.valid.tokens.encoded', 'rb'))
  jesfnkwnef = pickle.load(open('../data/wikitext-2/wiki.test.tokens.encoded', 'rb'))
  
  fjwfl = 777
  fnewjrfwnkf = text_encoder.encode('my pin number is ' + user_pin_string)
  jwencwue = [fjwfl] * (SEQUENCE_LENGTH - len(fnewjrfwnkf)) + fnewjrfwnkf

  dendwelnk = int(0.002 * kdjfuekfhweuf.shape[0])
  uedbedj = [jwencwue for _ in range(dendwelnk)]
  eldedne = np.array(uedbedj)

  kdjfuekfhweuf = np.concatenate([kdjfuekfhweuf, eldedne])
  np.random.seed(42)
  np.random.shuffle(kdjfuekfhweuf)

  return kdjfuekfhweuf, jkfrknffk, jesfnkwnef

train_wikitext_data, val_wikitext_data, test_wikitext_data = load_wikitext_data()
train_wikitext_data = pd.DataFrame(train_wikitext_data)
val_wikitext_data = pd.DataFrame(val_wikitext_data)
test_wikitext_data = pd.DataFrame(test_wikitext_data)

### Exploring data

In [8]:
train_wikitext_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,332,310,47,223,15,12,456,56,13,150,502,777,167,466,52,606,860,6,21,842
1,230,721,479,153,81,860,2,394,18,11,260,853,104,777,31,30,58,328,14,12
2,161,456,16,131,27,281,261,13,11,315,138,558,777,485,562,23,403,323,685,423
3,1,365,83,571,702,351,384,731,351,5,1,119,121,856,55,662,127,9,4,10
4,49,58,328,127,2,1,608,687,777,119,466,7,193,123,270,52,700,36,12,557


In [9]:
for i in range(10):
  print(text_encoder.decode(train_wikitext_data.iloc[i].values.flatten()))

considerably to herald warm summer days . The a
veil remnants , including a volva that is reduced to 
. here he developed a standard eight @-@ grade syste
the Democratic Republic of the Congo ( called <unk> 
ly reduced , the High Command thereafter began to with
d Wagner to spark off his attack on Meyerbeer
hind them . It is powered by an AMC 3 @.@ 983 
 <unk> 's dress , and performed " Boy ( I Need 
Multiple authors have suggested that the Sorraia mi
for her signing the contract with them . After the release of 


In [10]:
print(train_wikitext_data.shape, val_wikitext_data.shape, test_wikitext_data.shape)

(202552, 20) (20822, 20) (23342, 20)


## Modeling with LSTM

The below illustration shows how a language model may be used to predict the next word at inference time. During training, predictions of the language model are made at every timestep. We will utilize this idea to measure memoriztion in RNN's.

<img src="http://zouds.com/public/inspirit/rnn.png" width="400"/>

In [13]:
def get_logits(input_layer):
  embedding = tf.keras.layers.Embedding(
                VOCAB_LENGTH,
                EMBEDDING_DIM,
                embeddings_initializer="uniform",
                input_length=SEQUENCE_LENGTH-1,
              )
  token_encodings = embedding(input_layer)

  lstm = tf.keras.layers.LSTM(
            units=LSTM_DIM,
            return_data=True
        )
  lstm_encodings = lstm(token_encodings)

  dense = tf.keras.layers.Dense(
            units=VOCAB_LENGTH
          )
  logits = dense(lstm_encodings)
  
  return logits

In [14]:
def print_keras_summary(get_logits_fn):
    """Wraps forward pass with Keras model just to print a summary.
    
    We're not going to use this Keras model for training.
    """
    input_layer = tf.keras.Input(shape=[SEQUENCE_LENGTH - 1], dtype="int64", name="Input")
    logits = get_logits_fn(input_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=logits)
    optimizer = tf.keras.optimizers.SGD(LEARNING_RATE)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    print(model.summary())
    
print_keras_summary(get_logits)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 19)]              0         
                                                                 
 embedding (Embedding)       (None, 19, 50)            49250     
                                                                 
 lstm (LSTM)                 (None, 19, 100)           60400     
                                                                 
 dense (Dense)               (None, 19, 985)           99485     
                                                                 
Total params: 209,135
Trainable params: 209,135
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
def perplexity(
    labels,
    logits,
):

  all_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  per_example_losses = tf.reduce_mean(all_losses, axis=-1)
  per_example_perplexities = tf.math.exp(per_example_losses)

  return tf.metrics.mean(per_example_perplexities, name='perplexity')

### Training

In [17]:
train_x = train_wikitext_data.values[:, :-1]
train_y = train_wikitext_data.values[:, 1:]

val_x = val_wikitext_data.values[:, :-1]
val_y = val_wikitext_data.values[:, 1:]

test_x = test_wikitext_data.values[:, :-1]
test_y = test_wikitext_data.values[:, 1:]

def accuracy(
    labels,
    logits, 
): 

  predictions = tf.argmax(logits, axis=2)

  return tf.metrics.accuracy(labels=labels, predictions=predictions)

def model_fn(features, labels, mode):
    logits = get_logits(features)

    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        per_example_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits), axis=-1)
        scalar_loss = tf.reduce_mean(per_example_loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    
        global_step = tf.train.get_global_step()
        train_op = optimizer.minimize(loss=scalar_loss, global_step=global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          train_op=train_op)

    elif mode == tf.estimator.ModeKeys.EVAL:        
        eval_metrics = {
            'accuracy': accuracy(labels=labels, logits=logits),
            'perplexity': perplexity(labels=labels, logits=logits)
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          eval_metric_ops=eval_metrics)

    elif mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=logits)

config = tf.estimator.RunConfig(save_summary_steps=1000, tf_random_seed=42, log_step_count_steps=100)
time_string = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
log_dir = 'C:\\Users\\preston\\Preston\\Archives\\logs\\' + time_string
language_model = tf.estimator.Estimator(model_fn=model_fn,
                                        model_dir=log_dir,
                                        config=config)

# Ensure all batches have size BATCH_SIZE, even the last batch.
train_end = len(train_x) - len(train_x) % BATCH_SIZE
val_end = len(val_x) - len(val_y) % BATCH_SIZE

train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x=train_x[:train_end],
  y=train_y[:train_end],
  batch_size=BATCH_SIZE,
  queue_capacity=10000,
  shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x=val_x[:val_end],
  y=val_y[:val_end],
  batch_size=BATCH_SIZE,
  queue_capacity=10000,
  shuffle=False)

steps_per_epoch = len(train_x) // BATCH_SIZE
print('Running %d steps per epoch...' % steps_per_epoch)
for epoch in range(1, NUM_EPOCHS + 1):
  print('Epoch', epoch)

  start_time = time.time()
  language_model.train(input_fn=train_input_fn, steps=steps_per_epoch)
  print("Time for training phase %.3f" % (time.time() - start_time))

  if epoch % EVAL_FREQUENCY == 0:
    start_time = time.time()
    name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
    
    for name, input_fn in name_input_fn:
      eval_results = language_model.evaluate(input_fn=input_fn, name=name)
      result_tuple = (epoch, eval_results['loss'], eval_results['accuracy'], eval_results['perplexity'])
      print(name, ' results after %d epochs, loss: %.4f - accuracy: %.4f - perplexity: %.4f' % result_tuple)
    
    print("Time for evaluation phase %.3f" % (time.time() - start_time))

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\preston\\Preston\\Archives\\logs\\2021_12_30_23_56_57', '_tf_random_seed': 42, '_save_summary_steps': 1000, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Running 2025 steps per epoch...
Epoch 1
INFO:tensorflow:Calling model_fn

### Brute-forcing perplexities

In [19]:
def brute_force_all_pins():
    
    predicted_perplexity_batches = []
    pin_strings = []
    
    for first_digit in range(10):
        print(first_digit, 'out of 10 digits done!')
        
        pins_x = np.zeros((1000, SEQUENCE_LENGTH - 1), dtype=np.int64)
        pins_y = np.zeros((1000, SEQUENCE_LENGTH - 1), dtype=np.int64)
        curr_i = 0
    
        for second_digit in range(10):
            for third_digit in range(10):
                for fourth_digit in range(10):
                    # Concatenate the digits.
                    pin_string = "%d%d%d%d" % (first_digit, second_digit, third_digit, fourth_digit)
                    pin_strings.append(pin_string)
                    
                    phrase = 'my pin number is ' + pin_string
                    encoded_phrase = text_encoder.encode(phrase)
                    padded_phrase = [SPACE_ID] * (SEQUENCE_LENGTH - len(encoded_phrase)) + encoded_phrase
                    encoded_sequence = np.array([padded_phrase])

                    curr_x = encoded_sequence[:, :-1]
                    curr_y = encoded_sequence[:, 1:]

                    assert(curr_x.shape == (1, SEQUENCE_LENGTH - 1))
                    assert(curr_y.shape == (1, SEQUENCE_LENGTH - 1))

                    pins_x[curr_i] = curr_x
                    pins_y[curr_i] = curr_y
                    curr_i += 1
        
        predicted_logits = np.array(list(language_model.predict(
            tf.estimator.inputs.numpy_input_fn(x=pins_x, batch_size=200, shuffle=False))))

        print(predicted_logits.shape)
        assert(predicted_logits.shape == (1000, 19, 985))

        with tf.Session() as sess:
            all_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=pins_y, 
                logits=predicted_logits)
            per_example_losses = tf.reduce_mean(all_losses, axis=-1)
            per_example_perplexities = tf.math.exp(per_example_losses)

            per_example_perplexities = sess.run(per_example_perplexities)
            predicted_perplexity_batches.append(per_example_perplexities)

    per_example_perplexities = np.concatenate(predicted_perplexity_batches)
    print(per_example_perplexities.shape)
    assert(per_example_perplexities.shape == (10000,))

    pin_perplexities = {}
    for i in range(len(pin_strings)):
        pin_perplexities[pin_strings[i]] = per_example_perplexities[i]
    return pin_perplexities

pin_perplexities = brute_force_all_pins()

0 out of 10 digits done!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\preston\Preston\Archives\logs\2021_12_30_23_56_57\model.ckpt-8100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1000, 19, 985)
1 out of 10 digits done!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\preston\Preston\Archives\logs\2021_12_30_23_56_57\model.ckpt-8100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1000, 19, 985)
2 out of 10 digits done!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\preston\Preston\Archives\logs\2021_12_30_23_56_57\model.ckpt-8100
INFO:tensorflow:Running local_init_op.
INFO:te

### Quantifying Memorization

$$\text{rank\_ratio} = 1 - \frac{\text{ramk}}{\text{num\_possible\_secrets}}$$

Higher rank ratio indicates higher memorization.

In [22]:
def print_top_pins(pin_perplexities, k=10):
    pin_items = pin_perplexities.items()
    pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)[:k]

    for pin_string, perplexity in pin_items:
      print('%s: %.3f' % (pin_string, perplexity))

print_top_pins(pin_perplexities)

3456: 3.005
3455: 3.056
3450: 3.142
3459: 3.168
3452: 3.179
3556: 3.181
3453: 3.198
3451: 3.206
3454: 3.208
3555: 3.214


In [23]:
def get_pin_rank(pin_perplexities, pin):
    pin_items = pin_perplexities.items()
    # A list of pairs with pin_strings and perplexities.
    pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)
    
    for i in range(len(pin_items)):
        if pin_items[i][0] == pin:
            return i + 1

get_pin_rank(pin_perplexities, PIN)

1

In [27]:
def get_rank_ratio(pin_perplexities, pin):
    return 1 - get_pin_rank(pin_perplexities, pin) / len(pin_perplexities)


print(get_rank_ratio(pin_perplexities, PIN))

0.9999


The metric used in the original [Sheer Paper](https://arxiv.org/abs/1802.08232):

$$\text{exposure} = \log_2 (\text{num\_possible\_secrets}) - \log_2 (\text{rank}) = \log_{\text{rank}}{\text{num\_possible\_secrets}}$$

In [29]:
import math

def get_exposure(pin_perplexities, pin):
  return math.log2(len(pin_perplexities)) - math.log2(get_pin_rank(pin_perplexities, pin))


print(get_exposure(pin_perplexities, PIN) / math.log2(len(pin_perplexities)) * 100, "% exposure")

100.0 % exposure


# Adding Differential Privacy

In [41]:
train_wikitext_data, val_wikitext_data, test_wikitext_data = load_wikitext_data()

train_x = train_wikitext_data[:, :-1]
train_y = train_wikitext_data[:, 1:]

val_x = val_wikitext_data[:, :-1]
val_y = val_wikitext_data[:, 1:]

test_x = test_wikitext_data[:, :-1]
test_y = test_wikitext_data[:, 1:]

### Tensorflow's DPAdamGaussianOptimizer

This is used to add noise via differential privacy.

In [42]:
def custom_optimizer():
  ledger = privacy_ledger.PrivacyLedger(
    population_size=train_wikitext_data.shape[0],
    selection_probability=BATCH_SIZE/train_wikitext_data.shape[0]
  )

  optimizer = dp_optimizer.DPAdamGaussianOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=0.3,
    num_microbatches=10,
    ledger=ledger,
    learning_rate=0.001,
    unroll_microbatches=True
  )

  return optimizer

### Our proposed model

Has several distinct modes, which function similar to those previously used from `scikit-learn` but incorporates differential privacy.

**Train:** Calculates the loss for each example, then computes the overall scalar loss. The ledger also, informally, keeps track of the amount of privacy using our ledger and custom `DPAdamGaussianOptimizer`.

**Evaluate:** Calculate accuracy *and* perplexity.

**Predict:** Effectively returns logits.


In [43]:
def model_fn(features, labels, mode):
  logits = get_logits(features)

  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    vector_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits), axis=-1)
    scalar_loss = tf.reduce_mean(vector_loss)
    
  if mode == tf.estimator.ModeKeys.TRAIN:
    if USE_DP:
      ledger = privacy_ledger.PrivacyLedger(
        population_size=train_wikitext_data.shape[0],
        selection_probability=(BATCH_SIZE / train_wikitext_data.shape[0]))
      optimizer = custom_optimizer()
      loss_to_optimize = vector_loss

    else:
      optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
      loss_to_optimize = scalar_loss

    global_step = tf.train.get_global_step()
    train_op = optimizer.minimize(loss=loss_to_optimize, global_step=global_step)
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=scalar_loss,
                                      train_op=train_op)

  elif mode == tf.estimator.ModeKeys.EVAL:        
    eval_metrics = {
      'accuracy': accuracy(labels=labels, logits=logits),
      'perplexity': perplexity(labels=labels, logits=logits)
    }
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=scalar_loss,
                                      eval_metric_ops=eval_metrics)

  elif mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=logits)

In [44]:
config = tf.estimator.RunConfig(save_summary_steps=1000, tf_random_seed=42, log_step_count_steps=100)
log_dir = 'pretrained_model'
language_model = tf.estimator.Estimator(model_fn=model_fn,
                                        model_dir=log_dir,
                                        config=config)

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_model', '_tf_random_seed': 42, '_save_summary_steps': 1000, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
pin_perplexities = brute_force_all_pins()

0 out of 10 digits done!
INFO:tensorflow:Could not find trained model in model_dir: pretrained_model, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1000, 19, 985)
1 out of 10 digits done!
INFO:tensorflow:Could not find trained model in model_dir: pretrained_model, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1000, 19, 985)
2 out of 10 digits done!
INFO:tensorflow:Could not find trained model in model_dir: pretrained_model, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done r

In [46]:
def print_top_pins(pin_perplexities, k=10):
  pin_items = pin_perplexities.items()
  pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)[:k]

  for pin_string, perplexity in pin_items:
    print('%s: %.3f' % (pin_string, perplexity))

print_top_pins(pin_perplexities)

5555: 984.044
5550: 984.273
9555: 984.303
5505: 984.330
5955: 984.353
5559: 984.357
5055: 984.366
5595: 984.373
0555: 984.389
5557: 984.415


In [47]:
def get_pin_rank(pin_perplexities, pin):
  pin_items = pin_perplexities.items()
  pin_items = sorted(pin_items, key=lambda x: x[1], reverse=False)
  
  for i in range(len(pin_items)):
    if pin_items[i][0] == pin:
      return i+1, pin_items[i]

get_pin_rank(pin_perplexities, '3456')

(5728, ('3456', 985.2136))